In [34]:
# The following section is used to generate information about the datasets so these 
# variables can be initialised in the CNN Model. The information gathered here will
# also be used to determine an appropiate size of the training and testing datasets.
import os                                                   # Import os to change the information surrounding the enviroment 
from keras.preprocessing.image import ImageDataGenerator    # Create tensorflow image batches using data augmentation.
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
locationOfDataset = '/home/dmr/DatasetForExperiments/adware/'
batches = ImageDataGenerator().flow_from_directory(directory=locationOfDataset, target_size=(64,64), batch_size=321)

# Print information about Images 
imgs, labels = next(batches)
print("Number of Images and Size of Images: ",imgs.shape) # Images with width x length x depth
print("Number of Images and Families: ",labels.shape) # Lablels with batch_size, number of classes

Found 107 images belonging to 10 classes.
Number of Images and Size of Images:  (107, 64, 64, 3)
Number of Images and Families:  (107, 10)


In [35]:
# This section will cover the necessary steps used to split the scareware dataset
# into training and testing to evaluate the effectiveness of this convolutional neural 
# network model based on the scareware samples.
import numpy as np                                   # Provides the ability to use numerical data to convert the .dex files to greyscale images
import scipy as sp                                   # Provides the use of algorithms to be used 
from PIL import Image                                # Allow for processing of images

# Splitting the Dataset into Training and testing with a test size of 30% of overall 
# samples in the dataset. 
from sklearn.model_selection import train_test_split    # Used to split the dataset

# Split up the data set, 30% testing 70% training
trainX, testX, trainY, testY = train_test_split(imgs/255.,labels, test_size=0.3)

# Print the information regarding the training and testing sample sizes
#print("Size of Training Set: ", trainX.shape)                   # Print training set information
#print("Size of Testing Set: ",testX.shape)                      # Print testing set information
print("Number of Samples in Training Set: ",trainY.shape)   # Print training set information
print("Number of Samples in Testing Set: ",testY.shape)     # Print testing set information

Number of Samples in Training Set:  (74, 10)
Number of Samples in Testing Set:  (33, 10)


In [36]:
# The Convolutional Neural Network Model
# This section will encompass the aspects surrounding the convultional neural network
# which was used for the experimentation of this research. 
import keras                                            # Import keras
import tensorflow                                       # Import tensorflow
from keras.models import Sequential, Input, Model       # Used for CNN model regarding Models
from keras.layers import Dense, Dropout, Flatten        # Used for CNN model regarding Layers
from keras.layers import Conv2D, MaxPooling2D           # Used for CNN model regarding Layers
from tensorflow.keras.layers import BatchNormalization  # Layer used to normalise inputs
from tensorflow.keras.metrics import AUC                # Used estimate to Area Under Curve

noOfFamilies = 10 # Number of Families in the Sample

# Function used to create the layers of the convolutional neural network model. These parameters were
# constantly changed throughout testing in order to obtain the best possible results. Within this model 
# there are 10 main layers and each layers parameters are explained in detail. 
def CNNModel():
    CNNModel = Sequential()        # Sequential Model is used the experiments

    # Convolutional Layer: With relu Activation, 30 filters, and a kernel size of 3 x 3
    CNNModel.add(Conv2D(30, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))                # Layer 1 

    # Max Pooling Layer: With a pool size of 2 x 2
    CNNModel.add(MaxPooling2D(pool_size=(2, 2)))            # Layer 2
    
    # Convolutional Layer: 15 filters and kernel size of 3 x 3 
    CNNModel.add(Conv2D(15, (3, 3), activation='relu'))     # Layer 3
    
    # Max Pooling Layer: Pool size of 2 x 2 
    CNNModel.add(MaxPooling2D(pool_size=(2, 2)))            # Layer 4
    
    # Drop Out Layer: This layer used to drop 25% of neurons in the model .
    CNNModel.add(Dropout(0.25))                             # Layer 5
    
    # Flatten Layer
    CNNModel.add(Flatten())                                 # Layer 6
    
    # Dense/Fully Connected Layer : 128 Neurons using the relu activation function
    CNNModel.add(Dense(128, activation='relu'))             # Layer 7
    
    # Drop Out Layer: This layer used to drop 50% of neurons in the model.
    CNNModel.add(Dropout(0.5))                              # Layer 8
    
    # Dense/Fully Connected Layer : 50 Neurons using the relu activation function
    CNNModel.add(Dense(50, activation='relu'))              # Layer 9
    
    # Dense/Fully Connected Layer : number of families neurons, using the softmax activation function
    CNNModel.add(Dense(noOfFamilies, activation='softmax')) # Layer 10
    
    CNNModel.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return CNNModel     # Return the Convolutional Neural Network Model

In [37]:
# Measure the effectiveness of the model. This section is used to generate the results of the 
# corresponding convolutional neural network. For the purpose of this mode, there are three main 
# measurements which have to be declared. These are recall, preicision and f1. Listed below
# are the neccessary formulas which are required to obtain these results.
from keras import backend as K

# Recall Measurement
def recallMeasurement(testY, yPrediction):
    true_positives = K.sum(K.round(K.clip(testY * yPrediction, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(testY, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# Precision Measurement
def precisionMeasurement(testY, yPrediction):
    true_positives = K.sum(K.round(K.clip(testY * yPrediction, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(yPrediction, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# F1 Measurement
def f1Measurement(testY, yPrediction):
    precision = precisionMeasurement(testY, yPrediction)
    recall = recallMeasurement(testY, yPrediction)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [38]:
# Initialise the Model
CNNModel = CNNModel()

In [39]:
# Print the Information of the Model. This includes the information surrounding 
# the layers of the models as well as the number of total parameters
CNNModel.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 62, 62, 30)        840       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 31, 31, 30)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 29, 29, 15)        4065      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 15)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 14, 14, 15)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 2940)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)              

In [40]:
trainY.shape                            # Returns the shape of the trainY array 
trainY_new = np.argmax(trainY, axis=1)  # Converts the trainY array to an array of integers 
trainY_new                              # Returns the array of integer labels

array([9, 3, 2, 3, 0, 2, 7, 0, 4, 4, 7, 7, 3, 3, 9, 0, 1, 0, 8, 6, 1, 9,
       1, 4, 6, 5, 4, 1, 6, 6, 5, 8, 2, 7, 5, 9, 4, 8, 4, 0, 2, 3, 9, 5,
       1, 1, 6, 5, 8, 5, 4, 0, 4, 3, 2, 4, 3, 7, 7, 6, 5, 4, 2, 2, 3, 5,
       0, 2, 3, 2, 5, 5, 6, 3])

In [41]:
# This section of the model imports the class_weight function from the sklearn.utils 
# library. This is used to compute the class weights for a multi-class classification problem.
# The weightOfModel computes the class weights based on the class distribution found in the 
# training set. The class_weight parameter is set to balanced which means that the class 
# weights are automatically adjusted in acorrdance with the information obtained from the 
# input data.

from sklearn.utils import class_weight # Used to compute the class weights for a multi-class classification system
weightOfModel = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(trainY_new),
                                                 y = trainY_new)

# This code is used to map each unique class label in the training set trainY_new variable. 
# This is then uses the class_weight variable initialised above. 
weightOfModel = {l:c for l,c in zip(np.unique(trainY_new), weightOfModel)}

In [42]:
# This is used to compile the Convolutional Neural network Model. It uses the adam optimiser, 
# computes loss using binary_crossentropy and then calculates the measurements listed before.
CNNModel.compile(optimizer='adam', 
                    loss='binary_crossentropy', 
                    metrics=[f1Measurement,precisionMeasurement, 
                    recallMeasurement,
                    AUC()])

In [43]:
# This section of the model is used to trained the convolutional neural network model. 
# This is achieved by using the fit() method from the Keras API of the TensorFlow library which 
# was imported previously. Here is where the number of epochs are declared for the model also
CNNModel.fit(trainX, trainY, validation_data=(testX, testY), 
                epochs=100,  
                class_weight=weightOfModel)

Epoch 1/100
3/3 [==============================] - 1s 162ms/step - loss: 0.5883 - f1Measurement: 0.0000e+00 - precisionMeasurement: 0.0000e+00 - recallMeasurement: 0.0000e+00 - auc_3: 0.5586 - val_loss: 0.3327 - val_f1Measurement: 0.0000e+00 - val_precisionMeasurement: 0.0000e+00 - val_recallMeasurement: 0.0000e+00 - val_auc_3: 0.6067
Epoch 2/100
3/3 [==============================] - 0s 50ms/step - loss: 0.3933 - f1Measurement: 0.0000e+00 - precisionMeasurement: 0.0000e+00 - recallMeasurement: 0.0000e+00 - auc_3: 0.5166 - val_loss: 0.3584 - val_f1Measurement: 0.0000e+00 - val_precisionMeasurement: 0.0000e+00 - val_recallMeasurement: 0.0000e+00 - val_auc_3: 0.6022
Epoch 3/100
3/3 [==============================] - 0s 53ms/step - loss: 0.3868 - f1Measurement: 0.0819 - precisionMeasurement: 0.2188 - recallMeasurement: 0.0505 - auc_3: 0.5396 - val_loss: 0.3241 - val_f1Measurement: 0.0000e+00 - val_precisionMeasurement: 0.0000e+00 - val_recallMeasurement: 0.0000e+00 - val_auc_3: 0.6182
Epo

3/3 [==============================] - 0s 50ms/step - loss: 0.2176 - f1Measurement: 0.2769 - precisionMeasurement: 0.9097 - recallMeasurement: 0.1635 - auc_3: 0.9339 - val_loss: 0.2651 - val_f1Measurement: 0.1579 - val_precisionMeasurement: 0.5000 - val_recallMeasurement: 0.0938 - val_auc_3: 0.8110
Epoch 27/100
3/3 [==============================] - 0s 51ms/step - loss: 0.2136 - f1Measurement: 0.4341 - precisionMeasurement: 0.7723 - recallMeasurement: 0.3073 - auc_3: 0.8939 - val_loss: 0.2691 - val_f1Measurement: 0.1750 - val_precisionMeasurement: 0.4375 - val_recallMeasurement: 0.1094 - val_auc_3: 0.7903
Epoch 28/100
3/3 [==============================] - 0s 51ms/step - loss: 0.1945 - f1Measurement: 0.5338 - precisionMeasurement: 0.9417 - recallMeasurement: 0.3768 - auc_3: 0.9271 - val_loss: 0.2726 - val_f1Measurement: 0.1905 - val_precisionMeasurement: 0.4000 - val_recallMeasurement: 0.1250 - val_auc_3: 0.7710
Epoch 29/100
3/3 [==============================] - 0s 56ms/step - loss: 0

3/3 [==============================] - 0s 64ms/step - loss: 0.0741 - f1Measurement: 0.8700 - precisionMeasurement: 0.8899 - recallMeasurement: 0.8516 - auc_3: 0.9918 - val_loss: 0.2964 - val_f1Measurement: 0.1964 - val_precisionMeasurement: 0.2292 - val_recallMeasurement: 0.1719 - val_auc_3: 0.7980
Epoch 53/100
3/3 [==============================] - 0s 59ms/step - loss: 0.0779 - f1Measurement: 0.8511 - precisionMeasurement: 0.9208 - recallMeasurement: 0.7932 - auc_3: 0.9814 - val_loss: 0.3030 - val_f1Measurement: 0.1864 - val_precisionMeasurement: 0.2037 - val_recallMeasurement: 0.1719 - val_auc_3: 0.7997
Epoch 54/100
3/3 [==============================] - 0s 57ms/step - loss: 0.0682 - f1Measurement: 0.9106 - precisionMeasurement: 0.9444 - recallMeasurement: 0.8792 - auc_3: 0.9961 - val_loss: 0.3107 - val_f1Measurement: 0.2037 - val_precisionMeasurement: 0.2500 - val_recallMeasurement: 0.1719 - val_auc_3: 0.8033
Epoch 55/100
3/3 [==============================] - 0s 55ms/step - loss: 0

3/3 [==============================] - 0s 51ms/step - loss: 0.0455 - f1Measurement: 0.9128 - precisionMeasurement: 0.9128 - recallMeasurement: 0.9128 - auc_3: 0.9966 - val_loss: 0.3335 - val_f1Measurement: 0.6930 - val_precisionMeasurement: 0.7200 - val_recallMeasurement: 0.6719 - val_auc_3: 0.7611
Epoch 79/100
3/3 [==============================] - 0s 53ms/step - loss: 0.0563 - f1Measurement: 0.8911 - precisionMeasurement: 0.9041 - recallMeasurement: 0.8792 - auc_3: 0.9943 - val_loss: 0.3313 - val_f1Measurement: 0.1930 - val_precisionMeasurement: 0.2200 - val_recallMeasurement: 0.1719 - val_auc_3: 0.7581
Epoch 80/100
3/3 [==============================] - 0s 52ms/step - loss: 0.0530 - f1Measurement: 0.8572 - precisionMeasurement: 0.8793 - recallMeasurement: 0.8372 - auc_3: 0.9950 - val_loss: 0.3350 - val_f1Measurement: 0.1864 - val_precisionMeasurement: 0.2037 - val_recallMeasurement: 0.1719 - val_auc_3: 0.7512
Epoch 81/100
3/3 [==============================] - 0s 50ms/step - loss: 0

In [44]:
# Evaluates the trained  model on the test data and returns the values of the  metrics
scores = CNNModel.evaluate(testX, testY) 
print("--------------------------------------------------------------------------------------------------------------------")

# Evaluates the trained model on the test data and declares the loss, accuracy 
# f1_score, precision and recall
loss, accuracy, f1_score, precision, recall = CNNModel.evaluate(testX, testY)
print("--------------------------------------------------------------------------------------------------------------------")

# Used to Print the Final CNN Accuracy of the model
print("Final Results for The CNN Model")
print("F1 Accuracy of: {:.2f}".format(scores[1]),'%')    # Print Accuracy of Model
print("With a Loss of: {:.2f}".format(scores[0]),'%')    # Print Loss of Model

2/2 [==============================] - 0s 3ms/step - loss: 0.3658 - f1Measurement: 0.7097 - precisionMeasurement: 0.7167 - recallMeasurement: 0.7031 - auc_3: 0.7414
--------------------------------------------------------------------------------------------------------------------
2/2 [==============================] - 0s 3ms/step - loss: 0.3658 - f1Measurement: 0.7097 - precisionMeasurement: 0.7167 - recallMeasurement: 0.7031 - auc_3: 0.7414
--------------------------------------------------------------------------------------------------------------------
Final Results for The CNN Model
F1 Accuracy of: 0.71 %
With a Loss of: 0.37 %
